In [2]:
import pandas as pd
import numpy as np
import os
os.environ['PROJ_LIB'] = '/data/keeling/a/sshu3/anaconda2/share/proj'
from mpl_toolkits.basemap import Basemap, cm, shiftgrid,maskoceans
import matplotlib.pyplot as plt
from netCDF4 import Dataset as NetCDFFile
import C14preproc as prep
import pylab
import C14utils
import matplotlib
import socplot_lib as socplt
import isamcalc_lib as isam
import SOCtools as soc


In [ ]:
isam.latlon_2_idx(39.3232, -86.4131)

In [12]:
# =====================================================================
#  Plot the simulated GPP 
#  Compare with the Fluxnet observation
# =====================================================================
respath = '/data/jain1/c/sshu3/SBGC/runs/gpp_fluxnet/' 
obspath = '/data/jain1/c/sshu3/SBGC/runs/gpp_fluxnet/obs/'

site =  {
#     "CA-Ca1" : 0,
#     "CA-Ca2" : 1,
#     "CA-Gro" : 2,
    "CA-Let" : 3,
#     "CA-Mer" : 4,
#     "CA-NS1" : 5,
#     "CA-Oas" : 6,
#     "CA-Obs" : 7,
#     "CA-Ojp" : 8,
#     "CA-Qfo" : 9,
#     "CA-SJ1" : 10,
#     "CA-SJ2" : 11,
#     "CA-TP4" : 12,
#     "LBA-Ban" : 13,
#     "LBA-Fns" : 14,
#     "LBA-Km34" : 15,
#     "LBA-Km67" : 16,
#     "LBA-Km77" : 17,
#     "LBA-Km83" : 18,
#     "LBA-Pdg" : 19,  
#     "LBA-Rja" : 20,
#     "US-ARM" : 21,
#     "US-Atq" : 22,
#     "US-Brw" : 23,
#     "US-Dk3" : 24,
#     "US-Ho1" : 25,
#     "US-Los" : 26,
#     "US-Me2" : 27,
#     "US-Me5" : 28,
#     "US-NR1" : 29,
#     "US-SO2" : 30,
#     "US-Shd" : 31,
#     "US-Syv" : 32,
#     "US-Ton" : 33,
    "US-Var" : 34,
#     "US-WCr" : 35,
#     "US-Me3" : 36,
}

siteyear =  [7,4,1,8,6,4,8,5,5,1,3,2,3,1,3,3,3,4,3,3,3,5,2,1,7,9,5,3,4,7,7,4,4,4,5,7,1]

for siteid, ind in site.items():
    print(siteid)
    fname='/site_daily_2_pft.txt'
    mod_fname=respath+siteid+fname
    data = pd.read_csv(mod_fname,delim_whitespace=True,header=None)
    sz = np.shape(data)
    mod_gpp = data.values[(sz[0]-siteyear[ind]*365):sz[0],2]
    # Aggregate to annual
    sz = np.shape(mod_gpp)
    lenz = sz[0]/365
    mod_annual = np.zeros((lenz))
    for i in np.arange(0,lenz):
        mod_annual[i] = np.nansum(mod_gpp[(i*365):(i*365+365)]) 
    mod_annual[mod_annual<50] = np.float('nan')

    obs_fname=obspath+siteid+'_GPP.txt'
    data = pd.read_csv(obs_fname,delim_whitespace=True)
    sz = np.shape(data)
    obsdata = data.values[:,2]
    obsdata[obsdata<-100] = np.float('nan')
    obsdata = obsdata*0.0216
    # Aggregate to daily
    lenz = sz[0]/48
    obs_gpp = np.zeros((lenz))
    for i in np.arange(0,lenz):
        obs_gpp[i] = np.nansum(obsdata[(i*48):(i*48+48)]) 
    # Aggregate to annual
    sz = np.shape(obs_gpp)
    lenz = sz[0]/365
    obs_annual = np.zeros((lenz))
    for i in np.arange(0,lenz):
        obs_annual[i] = np.nansum(obs_gpp[(i*365):(i*365+365)]) 
    obs_annual[obs_annual<50] = np.float('nan')

    print('obs:')
    print(np.nanmean(obs_annual))
    print('mod:')
    print(np.nanmean(mod_annual))

US-Var
obs:
714.7744776
mod:
492.64055040000005
CA-Let
obs:
nan
mod:
314.825322875


/data/keeling/a/sshu3/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: Mean of empty slice


In [ ]:
1e-6*12*1800

In [ ]:
plt.plot(mod_gpp)
#plt.plot(obs_gpp)

In [ ]:
# Exmaine Jmax25
trop = 288.0
tlef_set = np.arange(273.0,320.0) 
rgas = 8.314
jmax = np.zeros((len(tlef_set)))
jmax25 = 450.0
i = 0
for tlef in tlef_set:
    jmax[i] = np.exp( 17.0e3 * (tlef - trop) / (rgas*trop*tlef) ) \
             * ( 1.0 + np.exp( (710.*trop-220.0e3)/(rgas*trop) ) ) \
             / ( 1.0 + np.exp( (710.*tlef-220.0e3)/(rgas*tlef) ) )
    i = i+1


In [ ]:
fig = plt.figure(figsize=(5,7))
plt.plot(tlef_set-273, jmax)

In [ ]:
# 35degC
t35 = np.array([4.99, 9.98, 14.96, 19.93, 24.97, 29.94, 34.91, 39.86, 44.94]) + 273.16
j35 = np.array([0.31, 0.57, 0.74, 0.83, 0.85, 0.88, 0.92, 0.75, 0.25])
# 25degC
t25 = np.array([5.0, 9.84, 14.96, 19.94, 24.91, 29.95, 34.9, 39.92, 44.93]) + 273.16
j25 = np.array([0.51, 0.72, 0.86, 0.95, 0.99, 0.94, 0.82, 0.52, 0.1])
# 15degC
t15 = np.array([4.93, 9.99, 14.81, 20.00, 24.97, 29.93, 34.96, 39.98, 45.0]) + 273.16
j15 = np.array([0.55, 0.75, 0.87, 0.82, 0.81, 0.75, 0.62, 0.32, 0.1])

# Parabolic curve regression
[a2, a1, a0] = np.polyfit(t35,j35,2)
jreg35 = np.zeros((len(j35)))
idx = 0
for i in t35:
    jreg35[idx] = a2*i*i + a1*i + a0
    idx = idx + 1
    
print(a2, a1, a0)

[a2, a1, a0] = np.polyfit(t25,j25,2)
jreg25 = np.zeros((len(j25)))
idx = 0
for i in t25:
    jreg25[idx] = a2*i*i + a1*i + a0
    idx = idx + 1

print(a2, a1, a0)
    
[a2, a1, a0] = np.polyfit(t15,j15,2)
jreg15 = np.zeros((len(j15)))
idx = 0
for i in t15:
    jreg15[idx] = a2*i*i + a1*i + a0
    idx = idx + 1
    
print(a2, a1, a0)

In [ ]:
plt.scatter(t15, j15)
plt.plot(t15, jreg15)

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
def func(x, peak, trop, shift):
    return np.exp(peak*1e3*(x-trop)/(8.314*trop*x)) * (1.0+np.exp((710.*trop-shift*1e3)/(8.314*trop))) / (1.0+np.exp((710.*x-shift*1e3)/(8.314*x)))

popt, pcov = curve_fit(func, t35, j35, bounds=([10.0, 278.0, 180.0], [30.0, 318.0, 250.0]))

jreg35 = np.zeros((len(j35)))
idx = 0
for i in t35:
    jreg35[idx] = func(i, *popt)
    idx = idx + 1

In [ ]:
popt

In [ ]:
plt.scatter(t35, j35)
plt.plot(t35, jreg35)

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
def func(x, peak, trop, shift):
    return np.exp(peak*1e3*(x-trop)/(8.314*trop*x)) * (1.0+np.exp((710.*trop-shift*1e3)/(8.314*trop))) / (1.0+np.exp((710.*x-shift*1e3)/(8.314*x)))

popt, pcov = curve_fit(func, t15, j15, bounds=([10.0, 258.0, 80.0], [50.0, 318.0, 350.0]))

jreg15 = np.zeros((len(j15)))
idx = 0
for i in t15:
    jreg15[idx] = func(i, *popt)
    idx = idx + 1

In [ ]:
popt

In [ ]:
popt=[ 19.5, 298.05677241, 218.05376074]
jreg15 = np.zeros((len(j15)))
idx = 0
for i in t15:
    jreg15[idx] = func(i, *popt)
    idx = idx + 1

In [ ]:
plt.scatter(t15, j15)
plt.plot(t15, jreg15)

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
def func(x, peak, trop, shift):
    return np.exp(peak*1e3*(x-trop)/(8.314*trop*x)) * (1.0+np.exp((710.*trop-shift*1e3)/(8.314*trop))) / (1.0+np.exp((710.*x-shift*1e3)/(8.314*x)))

popt, pcov = curve_fit(func, t25, j25, bounds=([10.0, 278.0, 180.0], [30.0, 318.0, 250.0]))

jreg25 = np.zeros((len(j25)))
idx = 0
for i in t25:
    jreg25[idx] = func(i, *popt)
    idx = idx + 1

In [ ]:
popt

In [ ]:
plt.scatter(t25, j25)
plt.plot(t25, jreg25)

In [ ]:
# Example of curve optimization
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def func(x, a, b, c):
    return a * np.exp(-b * x) + c

xdata = np.linspace(0, 4, 50)
y = func(xdata, 2.5, 1.3, 0.5)
np.random.seed(1729)
y_noise = 0.2 * np.random.normal(size=xdata.size)
ydata = y + y_noise
plt.plot(xdata, ydata, 'b-', label='data')

popt, pcov = curve_fit(func, xdata, ydata)

plt.plot(xdata, func(xdata, *popt), 'r-', \
        label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

popt, pcov = curve_fit(func, xdata, ydata, bounds=(0, [3., 1., 0.5]))
plt.plot(xdata, func(xdata, *popt), 'g--', \
        label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()


In [ ]:

# plt.plot(t35,j35)
# plt.plot(t35,jreg)

# plt.plot(t25,j25)
# plt.plot(t25,jreg)

plt.plot(t15,j15)
plt.plot(t15,jreg)

In [ ]:
fpath = '/data/jain1/c/sshu3/SBGC/runs/oneflux/gpp1.txt'
data = pd.read_csv(fpath,delim_whitespace=True,header=None)
data[data<0] = np.float('nan')
sz = np.shape(data)
lenz = sz[0]/365
obs_gpp = np.zeros((lenz))
for i in np.arange(0,lenz):
    obs_gpp[i] = np.nansum(data[(i*365):(i*365+365)]) 
plt.plot(obs_gpp)

In [ ]:
fpath = '/data/jain1/c/sshu3/SBGC/runs/oneflux/gpp2.txt'
data = pd.read_csv(fpath,delim_whitespace=True,header=None)
data[data<0] = np.float('nan')
data = data*0.0216
sz = np.shape(data)
lenz = sz[0]/48
obs_gpp = np.zeros((lenz))
for i in np.arange(0,lenz):
    obs_gpp[i] = np.nansum(data[(i*48):(i*48+48)])  
plt.plot(obs_gpp)

In [ ]:
fpath = '/data/jain1/c/sshu3/SBGC/runs/oneflux/gpp3.txt'
data = pd.read_csv(fpath,delim_whitespace=True,header=None)
data[data<0] = np.float('nan')
data = data*0.0216
sz = np.shape(data)
lenz = sz[0]/48
obs_gpp = np.zeros((lenz))
for i in np.arange(0,lenz):
    obs_gpp[i] = np.nansum(data[(i*48):(i*48+48)])  
plt.plot(obs_gpp)

In [ ]:
fpath = '/data/jain1/c/sshu3/SBGC/runs/oneflux/gpp4.txt'
data = pd.read_csv(fpath,delim_whitespace=True,header=None)
data[data<0] = np.float('nan')
data = data*0.0216
sz = np.shape(data)
lenz = sz[0]/48
obs_gpp = np.zeros((lenz))
for i in np.arange(0,lenz):
    obs_gpp[i] = np.nansum(data[(i*48):(i*48+48)])  
plt.plot(obs_gpp)

In [ ]:
plt.figure(figsize=(10,6))
plt.xlim([0,600])
plt.ylim([0,600])
plt.xlabel('SRDB NPP ($gC/m^2$)', fontsize=20)
plt.ylabel('ISAM-1D NPP ($gC/m^2$)', fontsize=20)
#plt.scatter(obs.NPP.as_matrix()[lc_code==3], isam_1d_res[lc_code==3], color='orange')
#plt.scatter(obs.NPP.as_matrix()[lc_code==4], isam_1d_res[lc_code==4], color='red')
plt.scatter(obs.NPP_adj.as_matrix()[lc_code==5], isam_1d_res[lc_code==5], color='green')
#plt.scatter(obs.NPP.as_matrix()[lc_code==20], isam_1d_res[lc_code==20], color='cyan')
#plt.scatter(obs.NPP.as_matrix()[lc_code==7], isam_1d_res[lc_code==7], color='blue')
#plt.scatter(obs.NPP.as_matrix()[lc_code==8], isam_1d_res[lc_code==8], color='black')
#plt.scatter(obs.NPP.as_matrix()[lc_code==12], isam_1d_res[lc_code==12], color='pink')
#plt.legend(['Temperate_evergreen', 'Temperate_deciduous', 'Boreal_evergreen', 'Boreal_deciduous', 'Grassland', 'Shrubland', 'Cropland'])
plt.savefig('./figures/fig_srdb_1d_npp.png')

In [ ]:
obs.NPP.as_matrix()[lc_code==5]

In [ ]:
isam_1d_res[lc_code==5]

In [ ]:
plt.figure(figsize=(10,6))
plt.xlim([0,1200])
plt.ylim([0,1200])
plt.xlabel('SRDB NPP ($gC/m^2$)', fontsize=20)
plt.ylabel('ISAM-0D NPP ($gC/m^2$)', fontsize=20)
plt.scatter(obs.NPP.as_matrix()[lc_code==3], isam_0d_res[lc_code==3], color='orange')
plt.scatter(obs.NPP.as_matrix()[lc_code==4], isam_0d_res[lc_code==4], color='red')
plt.scatter(obs.NPP.as_matrix()[lc_code==5], isam_0d_res[lc_code==5], color='green')
plt.scatter(obs.NPP.as_matrix()[lc_code==20], isam_0d_res[lc_code==20], color='cyan')
plt.scatter(obs.NPP.as_matrix()[lc_code==7], isam_0d_res[lc_code==7], color='blue')
plt.scatter(obs.NPP.as_matrix()[lc_code==8], isam_0d_res[lc_code==8], color='black')
plt.scatter(obs.NPP.as_matrix()[lc_code==12], isam_0d_res[lc_code==12], color='pink')
plt.legend(['Temperate_evergreen', 'Temperate_deciduous', 'Boreal_evergreen', 'Boreal_deciduous', 'Grassland', 'Shrubland', 'Cropland'])
plt.savefig('./figures/fig_srdb_0d_npp.png')

In [ ]:
# Boxplot of the total SOC in the first 1m
# Shijie: Shall filter out peat before plotting!
srdb = obs.Rh_annual.as_matrix().astype('float')
srdb[np.isnan(isam_1d_res)] = np.float('nan')
#data_for_bplot = [isam_0d_res, isam_1d_res, obs.Rh_annual.as_matrix()]

data_for_bplot = [isam_0d_res[~np.isnan(isam_0d_res)], isam_1d_res[~np.isnan(isam_1d_res)], srdb[~np.isnan(srdb)]]

# multiple box plots on one figure
plt.figure(figsize=(24,15))
boxprops = dict(linestyle='-', linewidth=4, color='darkgoldenrod')
flierprops = dict(marker='o', markerfacecolor='green', markersize=18,
                  linestyle='none')
medianprops = dict(linestyle='-.', linewidth=2.5, color='firebrick')
meanprops = dict(marker='^', markerfacecolor='blue', markersize=18,
                  linestyle='none')
#meanlineprops = dict(linestyle='--', linewidth=2.5, color='purple')

plt.boxplot(data_for_bplot, boxprops=boxprops, flierprops=flierprops, \
           medianprops=medianprops, meanprops=meanprops, showmeans=True) #, meanlineprops=meanlineprops)
plt.xticks([1, 2, 3], ['ISAM-0D', 'ISAM-1D', 'OBS (SRDB)'], fontsize=30)
plt.yticks([0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000], fontsize=30)
#plt.xlim((0, 9))
#plt.ylim((-500, 500))
#plt.xlabel('xlabel')
plt.ylabel('Heterotrophic Respiration ($gC/m^2$)', fontsize=30)
plt.grid(color='gray', which='major', axis='both', alpha=0.3)
plt.minorticks_on()
#plt.axhline((np.nanmean(data_for_bplot[2])+np.nanmean(data_for_bplot[3]))/2, color="gray", linestyle= 'dashdot')
plt.tick_params(axis='x',which='major',direction='in',length=8, width=2)
plt.tick_params(axis='x',which='minor',bottom='off')
plt.tick_params(axis='y',which='major',direction='in',length=8, width=2)
plt.tick_params(axis='y',which='minor',direction='in',length=3)
plt.tick_params(axis="x", which="major", top=True)
plt.tick_params(axis="y", which="both", right=True)
plt.tick_params(axis='x', pad=20)

plt.show()
#plt.savefig('./figures/fig_rh.png')

In [ ]:
# NPP vs latitudes (Photo-period)
ppp = ~np.isnan(isam_1d_res[lc_code==5])
lats_npp = obs.Latitude.as_matrix()[lc_code==5][ppp]
lons_npp = obs.Longitude.as_matrix()[lc_code==5][ppp]
plt.scatter(lats_npp, isam_1d_res[lc_code==5][ppp], color='red')
plt.scatter(lats_npp, obs.NPP.as_matrix()[lc_code==5][ppp], color='green')
plt.savefig('Photoperiod.png')

In [ ]:
# NPP vs temperature
ppp = ~np.isnan(isam_1d_res[lc_code==5])
#tas_npp = obs.MAT.as_matrix()[lc_code==5][ppp]
plt.scatter(isam_1d_tas[lc_code==5][ppp], isam_1d_res[lc_code==5][ppp], color='red')
plt.scatter(isam_1d_tas[lc_code==5][ppp], obs.NPP.as_matrix()[lc_code==5][ppp], color='green')
plt.legend(['ISAM simulation', 'Obs'])
plt.savefig('AirTemperature.png')

In [ ]:
lats_npp

In [ ]:
isam_1d_res[lc_code==5][ppp]

In [ ]:
obs.NPP.as_matrix()[lc_code==5][ppp]

In [ ]:
# ===============================================
# plot the locations of all samples
# ===============================================

fig = plt.figure(figsize=(20,12))
#ax = fig.add_axes([0.05,0.05,0.9,0.9])
# Polar Lambert Azimuthal Projection
m = Basemap(projection='npstere',boundinglat=45,lon_0=0,resolution='l')
#m = Basemap(projection='nplaea',boundinglat=40.,lon_0=270.,resolution='l')
m.drawcoastlines()
m.fillcontinents(color='coral',lake_color='aqua',zorder=0)
# draw parallels and meridians.
m.drawparallels(np.arange(45.,90.,15.))
m.drawmeridians(np.arange(-180.,180.,20.))
m.drawmapboundary(fill_color='aqua')
#ax = plt.gca()

# # Overlay the permafrost status map
# lon_p,lat_p = np.meshgrid(lonnc,latnc) #Returns coordinate matrices from coordinate vectors
# x, y = m(lon_p,lat_p)

# # Create new cmap for permafrost
# cmap_name = 'Permafrost'
# colors = [(0.937, 0.965, 1), (0.282, 0.373, 0.537), (0.278, 0.478, 0.686), (0.278, 0.576, 0.922), (0.588, 0.757, 0.922),
#           (0.918, 0.886, 0.741)]
# n_bin = 6
# newcm = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bin)

# #djet.set_bad('white',1.)

# # cs1 = map.contourf(x,y,mdata,clevs,cmap=plt.cm.jet)
# # cs1 = map.contourf(x,y,ncvar_maize[:,:], [100], colors="black")
# cs = m.pcolormesh(x,y,p[:,:], cmap=newcm, edgecolors = 'None') # use log color scale,
#                                                       # omit this to use linear
#                                                      # color scale
# cbar = m.colorbar(cs,location='right',pad="5%",ticks=None,extend='neither')
# #plt.title('Maize harvested area fraction')
# cbar.set_ticks([])
# #cbar.ax.set_xticklabels(['', '', '', '', '', ''])
# #cbar.ax.tick_params(labelsize=18) 

# m.fillcontinents(color='grey',lake_color='#99ffff',zorder=0)
# First add Mishra's data
x, y = m(lons_npp, lats_npp)
m.scatter(x,y,20,marker='o',color='r',label='Boreal Evergreen')#,alpha=0.7)#,latlon=True)
# Then add He's data
plt.legend(scatterpoints=1,loc=1, prop={'size': 18})

#ax.set_title('Profile Sites')
plt.title(' ')
plt.show()

#fig.savefig('./Shu2018_Fig2.png')

In [ ]:
# =====================================================================
# Optional
# =====================================================================
# 2) The comparison of the accumulated SOC profile
obs_aggprof_avg = np.nanmean(obs_agg_prof, axis=0)
mod_aggprof_avg = np.nanmean(mod_agg_prof, axis=0)
obs_aggprof_std = np.nanstd(obs_isam*dz, axis=0)
mod_aggprof_std = np.nanstd(mod_isam*dz, axis=0)
obs_aggprof_avg[9] = float("nan")
obs_aggprof_std[9] = float("nan")
mod_aggprof_std[:] = 0.
obs_aggprof_std = obs_aggprof_std #/np.sqrt(146.)  # 146 is the sample size
mod_aggprof_std = mod_aggprof_std #/np.sqrt(146.)  # 146 is the sample size

soildp = z[0:8]*100
#tit = "Accumulated SOC profile"
tit = ""
path = "./figures/fig5/Um_SOC_profile.png"

kwargs = {'Xobs': obs_aggprof_avg[0:8], 'Yobs': soildp, 'Xmod': mod_aggprof_avg[0:8], 'Ymod': soildp,
          'Xobs_std': obs_aggprof_std[0:8], 'Xmod_std': mod_aggprof_std[0:8], 'tit': ' ', 'path': path}

# Figure with std as error bar
status = socplt.plot_obsvsmod_with_errbar(**kwargs)


In [ ]:
# =====================================================================
# Optional
# =====================================================================
# 3) The comparison of the accumulated SOC profile by different biomes
# Pick one biome and run the code once at a time
# 5 biomes in total:
# 7 - grassland
# 8 - shrub
# 9 - tundra
# 5 - boreal evergreen
# 20 - boreal deciduous
obs_agg_prof_pft = obs_agg_prof[mod_biome.index == 7]
mod_agg_prof_pft = 1.7*mod_agg_prof[mod_biome.index == 7]
obs_aggprof_avg = np.nanmean(obs_agg_prof_pft, axis=0)
mod_aggprof_avg = np.nanmean(mod_agg_prof_pft, axis=0)
obs_aggprof_std = np.nanstd(obs_isam[mod_biome.index == 7]*dz, axis=0)
mod_aggprof_std = np.nanstd(mod_isam[mod_biome.index == 7]*dz, axis=0)
obs_aggprof_avg[9] = float("nan")
obs_aggprof_std[9] = float("nan")
mod_aggprof_std[:] = 0.
obs_aggprof_std = obs_aggprof_std #/np.sqrt(146.)  # 146 is the sample size
mod_aggprof_std = mod_aggprof_std #/np.sqrt(146.)  # 146 is the sample size

soildp = z[0:8]*100
#tit = "Accumulated SOC profile"
tit = ""
path = "./SOC_grassland_profile.png"

status = socplt.plot_obsvsmod_with_errbar(obs_aggprof_avg[0:8], soildp, mod_aggprof_avg[0:8], soildp, obs_aggprof_std[0:8], mod_aggprof_std[0:8], tit, path)


In [ ]:
# =====================================================================
#  Plot the simulated 1-m integrated D14C profiles for Umakant's samples
#  Compare with the others?
# =====================================================================
# ==========================================
## Recalculate the Tau for 1-D ISAM model
# ==========================================
N_sample=197
fmod_pid = "caselist_197sites"
fmod_biome = "pftlist_197sites"
fmod = "isam_um_dc14.dat"
fmod_soc = "isam_um_soc.dat"

# Read in D14C from observation
data = pd.read_csv(fmod, delim_whitespace=True, header=None)
pid = pd.read_csv(fmod_pid, header=None, index_col=0)
data.index = pid.index

# Read in the SOC profile
dsoc = pd.read_csv(fmod_soc, delim_whitespace=True, header=None)
dsoc.index = pid.index
soc = dsoc.as_matrix()
frac = soc * np.float("nan")
for i in np.arange(0,7):
    frac[:,i] = soc[:,i]/(np.sum(soc[:,0:7], axis=1)+0.3*soc[:,7])
frac[:,7] = 0.3*soc[:,7]/(np.sum(soc[:,0:7], axis=1)+0.3*soc[:,7])
    
# Assign Nan value for the all the data with -9999.
data[data<-1200.] = float("nan")
# Get the C-weighted D14
d14c = np.nansum(frac * data, axis=1)
d14c[d14c==0.]=np.float("nan")

# Calculate tau
z, dz, zsoih = isam.get_isam_soildp(10)
# First get the C-weighted mean D14C for each profile
 
sampleyr = 2010 * np.ones(len(d14c))

tau, cost = C14utils.cal_tau(d14c, sampleyr, 1, 0)
# Not showing too extreme anomalies
tau[tau>=4.00000000e+03] = np.float("nan")
data['tau'] = pd.Series(tau[:,0], index=data.index)
data.nodedepth = z*100
#tau=np.loadtxt("isam_um_tau.csv")
ttt=tau.reshape(N_sample)

# ==========================================
## Recalculate the Tau for 0-D ISAM model
# ==========================================
fmod_pid = "caselist_197sites"
fmod_biome = "pftlist_197sites"
fmod = "isam_0d_dc14.dat"
fmod_soc = "isam_0d_soc.dat"

# Read in D14C from observation
data0d = pd.read_csv(fmod, delim_whitespace=True, header=None)
pid = pd.read_csv(fmod_pid, header=None, index_col=0)
data0d.index = pid.index

# Read in the SOC profile
dsoc = pd.read_csv(fmod_soc, delim_whitespace=True, header=None)
dsoc.index = pid.index
soc0d = dsoc.as_matrix()

# Assign Nan value for the all the data with -9999.
data0d[data0d<-1200.] = float("nan")
# Get the C-weighted D14
d14c0d = np.nansum(data0d, axis=1)
d14c0d[d14c0d==0.]=np.float("nan")

# Calculate tau
z, dz, zsoih = isam.get_isam_soildp(10)
# First get the C-weighted mean D14C for each profile
 
sampleyr = 2010 * np.ones(len(d14c0d[0:N_sample]))

tau0, cost = C14utils.cal_tau(d14c0d[0:N_sample], sampleyr, 1, 0)
# Not showing too extreme anomalies
tau0[tau0>=4.00000000e+03] = np.float("nan")
data['tau'] = pd.Series(tau0[:,0], index=data.index)
data.nodedepth = z*100
ttt0=tau0.reshape(N_sample)

# ==========================================
## Site level estimations of Tau
# ==========================================
# Schadel et al., 2014
tau_org = 98.95
tau_min = 2350.443
# Assume the same fraction of organic vs mineral
tau_comb = 0.5*tau_org + 0.5*tau_min 
ttt_o = [tau_org, tau_min, tau_comb]

# Knoblauch et al., 2013
tau_a = 0.26*0.0222+170.3*0.9778
tau_ana = 0.21*0.0064+2652*0.9936
# Assume the same fraction of organic vs mineral
tau_comb = 0.5*tau_a + 0.5*tau_ana 
ttt_o2 = [tau_a, tau_ana, tau_comb]

# Trumbore and Harden, 1997

tau_s1 = 62.24605043
tau_s2 = 1017.085308
#tau_s3 = 1068.201909
tau_s3 = 2485.23
#ttt_o3 = [tau_s1, tau_s2, tau_s3, tau_s4]
ttt_o3 = [tau_s1, tau_s2, tau_s3]

lit_all = ttt_o + ttt_o2 + ttt_o3
mean_lit = np.nanmean(lit_all)


data_bplot = [ttt[~np.isnan(ttt)], ttt0[~np.isnan(ttt0)], ttt_o, ttt_o2, ttt_o3]

# Plot boxplot for tau
plt.figure(figsize=(24,15))
boxprops = dict(linestyle='-', linewidth=4, color='darkgoldenrod')
flierprops = dict(marker='o', markerfacecolor='green', markersize=18,
                  linestyle='none')
medianprops = dict(linestyle='-.', linewidth=2.5, color='firebrick')
meanprops = dict(marker='^', markerfacecolor='blue', markersize=18,
                  linestyle='none')
plt.boxplot(data_bplot,  boxprops=boxprops, flierprops=flierprops, \
           medianprops=medianprops, meanprops=meanprops, showmeans=True)
plt.xticks([1,2,3,4,5], ['ISAM-1D', 'ISAM-0D', 'Schadel et al.\n 2014', \
                         'Knoblauch et al.\n 2013', 'Trumbore and \n Harden, 1997'], fontsize=30)
#plt.boxplot(data_bplot,  boxprops=boxprops, flierprops=flierprops, \
#           medianprops=medianprops, meanprops=meanprops, showmeans=True)
# Add the line representing the mean value
plt.grid(color='gray', which='major', axis='both', alpha=0.3)
plt.minorticks_on()
plt.tick_params(axis='x',which='major',direction='in',length=8, width=2)
plt.tick_params(axis='x',which='minor',bottom='off')
plt.tick_params(axis='y',which='major',direction='in',length=8, width=2)
plt.tick_params(axis='y',which='minor',direction='in',length=3)
plt.tick_params(axis="x", which="major", top=True)
plt.tick_params(axis="y", which="both", right=True)
plt.tick_params(axis='x', pad=20)
plt.ylabel('Turnover Time ($year$)', fontsize=30)
plt.axhline(mean_lit, color="gray", linestyle= 'dashdot')
plt.yticks([0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000], fontsize=30)
plt.text(0.65, 3600, "(b)", fontsize=32)

plt.show()
#plt.savefig('./figures/fig5/fig5b.png')


In [ ]:
np.nanmean(data_bplot[0])

In [ ]:
np.nanmean(data_bplot[1])

In [ ]:
frac[41,:]

In [ ]:
soc[41,:]